In [1]:
%matplotlib inline
# !apt-get update
# !apt-get install -y libsndfile1
# !pip install librosa


DCGAN Tutorial
==============

In [2]:
# !pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113

from __future__ import print_function

import os
import random
from glob import glob

import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
import torch.onnx
from torchinfo import summary
from torch.utils.data import Dataset

from pre_process import *

# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


Inputs
------

Let’s define some inputs for the run:

-  **dataroot** - the path to the root of the dataset folder. We will
   talk more about the dataset in the next section
-  **workers** - the number of worker threads for loading the data with
   the DataLoader
-  **batch_size** - the batch size used in training. The DCGAN paper
   uses a batch size of 128
-  **image_size** - the spatial size of the images used for training.
   This implementation defaults to 64x64. If another size is desired,
   the structures of D and G must be changed. See
   `here <https://github.com/pytorch/examples/issues/70>`__ for more
   details
-  **nc** - number of color channels in the input images. For color
   images this is 3
-  **nz** - length of latent vector
-  **ngf** - relates to the depth of feature maps carried through the
   generator
-  **ndf** - sets the depth of feature maps propagated through the
   discriminator
-  **num_epochs** - number of training epochs to run. Training for
   longer will probably lead to better results but will also take much
   longer
-  **lr** - learning rate for training. As described in the DCGAN paper,
   this number should be 0.0002
-  **beta1** - beta1 hyperparameter for Adam optimizers. As described in
   paper, this number should be 0.5
-  **ngpu** - number of GPUs available. If this is 0, code will run in
   CPU mode. If this number is greater than 0 it will run on that number
   of GPUs




In [3]:
# Number of workers for dataloader
workers = 0

# Batch size during training
batch_size = 64

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 51

# Number of channels in the training images. For color images this is 3
nc = 201

# Size of z latent vector (i.e. size of generator input)
nz = 201

# Size of feature maps in generator
ngf = 32

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 100

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

In [4]:
train_data = glob("data/train/*.wav")
test_data = glob("data/test/*.wav")
unlabel_data = glob("data/unlabeled/*.wav")

train_df = pd.read_csv("data/train_data.csv")
test_df = pd.read_csv("data/test_data.csv")

In [5]:
# Decide which device we want to run on
if torch.cuda.is_available(): device = "cuda:0"
elif torch.has_mps: device = "mps"
else: device = "cpu"
device

'cuda:0'

In [6]:
class DataGen(Dataset):
    def __init__(self, data, df, is_train=True, only_img=False):
        self.x_data = data
        if not is_train:
            self.x_df = df
            pass
        else:
            self.x_df = np.array(df.iloc[:, :6])
            self.y = df["covid19"]
        self.is_train = is_train
        self.only_img = only_img

    def __getitem__(self, index):
        if self.is_train:
            try:
                if self.only_img: return zero_pad(mfcc(get_melspectogram(self.x_data[index]))).view(201, 20, 5), self.y[index]
                else: return zero_pad(mfcc(get_melspectogram(self.x_data[index])).view(201, 20, 5), self.x_df[index]), self.y[index]
            except:
                if self.only_img: return zero_pad(mfcc(get_melspectogram(self.x_data[index+1]))).view(201, 20, 5), self.y[index+1]
                else: return zero_pad(mfcc(get_melspectogram(self.x_data[index+1])).view(201, 20, 5), self.x_df[index+1]), self.y[index+1]
        else:
            if self.only_img: return torch.tensor(self.x_data[index]).view(201, 20, 5)
            else: return torch.tensor(self.x_data[index]).view(201, 20, 5), torch.tensor(self.x_df[index])

    def __len__(self):
        return len(self.x_data)

In [7]:
dataset = DataGen(data=train_data, df=train_df, only_img=True)
test_dataset = DataGen(data=test_data, df=test_df, is_train=False, only_img=True)

dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)
test_dataloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,
                                             shuffle=True, num_workers=workers)
# next(iter(dataloader))

In [8]:
# dataiter = iter(dataloader)
# dataiter.__next__()
next(iter(dataloader))[0].shape

K:\Programming\covid_audio\pre_process.py:55: FutureWarning: Pass y=[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.7774053e-09
   1.0156314e-05 2.4200200e-04]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.3046990e-05
   3.9981052e-05 3.3266447e-04]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.1256188e-03
   5.4382556e-03 3.5404158e-04]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.4745150e-17
   7.7195195e-17 7.1805056e-13]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 3.9166803e-18
   3.4369212e-17 7.1862563e-13]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 4.8789098e-19
   4.8789098e-19 7.1117613e-13]]] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.feature.mfcc(np.array(audio), sr=samplerate, n_mels=128)
G:\Programdata\envs\covid_audio\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=912
  return f(*args, **kwargs)
K:\Pro

torch.Size([64, 201, 20, 5])

In [9]:
# next(iter(dataset))[0].shape

# audio_shape = [audio[0].shape for audio in dataloader]
# pd.DataFrame(audio_shape).unique

# get_melspectogram(train_data[0]).shape

val = torch.Tensor(1, 201, 20, 10)
zero_pad(val).shape

# zero_pad(mfcc(get_melspectogram(train_data[0]))).view(1, 201, 20, 5).shape

torch.Size([1, 201, 5, 10])

Implementation
--------------

With our input parameters set and the
 prepared, we can now get
into the implementation. We will start with the weight initialization
strategy, then talk about the generator, discriminator, loss functions,
and training loop in detail.

Weight Initialization
~~~~~~~~~~~~~~~~~~~~~

From the DCGAN paper, the authors specify that all model weights shall
be randomly initialized from a Normal distribution with mean=0,
stdev=0.02. The ``weights_init`` function takes an initialized model as
input and reinitializes all convolutional, convolutional-transpose, and
batch normalization layers to meet this criteria. This function is
applied to the models immediately after initialization.




In [10]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [11]:
# Generator Code

class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu

        # input is Z, going into a convolution
        self.convTrans1 = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 1, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True)
        )

        # state size. (ngf*8) x 4 x 4
        self.convTrans2 = nn.Sequential(
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 1, 2, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True)
        )

        # state size. (ngf*4) x 8 x 8
        self.convTrans3 = nn.Sequential(
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 1, 2, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True)
        )

        # state size. (ngf*2) x 16 x 16
        self.convTrans4 = nn.Sequential(
            nn.ConvTranspose2d(ngf * 2, ngf, 1, 2, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True)
        )

        # state size. (ngf) x 32 x 32
        self.convTrans5 = nn.Sequential(
            nn.ConvTranspose2d(ngf, nc, (20, 5), 2, bias=False)
        )

        self.convTrans = []
        for i in range(20):
            self.convTrans.append(nn.ConvTranspose2d(ngf, ngf, 1, 1, bias=False))
            self.convTrans.append(nn.BatchNorm2d(ngf))
            self.convTrans.append(nn.ReLU(True))

        self.convTrans = nn.ModuleList(self.convTrans)

    def forward(self, input):
        out = self.convTrans1(input)
        out = self.convTrans2(out)
        out = self.convTrans3(out)
        out = self.convTrans4(out)
        for i in range(20):
            out = self.convTrans[i](out)
        out = self.convTrans5(out)
        return out

Now, we can instantiate the generator and apply the ``weights_init``
function. Check out the printed model to see how the generator object is
structured.




In [12]:
# Create the generator
netG = Generator(ngpu).to(device)

# Handle multi-gpu if desired
if (device == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))

# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.02.
netG.apply(weights_init)

# Print the model
# print(netG)
summary(netG, (batch_size, nz, 20, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Generator                                [64, 201, 324, 69]        631,168
├─Sequential: 1-1                        [64, 256, 20, 5]          --
│    └─ConvTranspose2d: 2-1              [64, 256, 20, 5]          51,456
│    └─BatchNorm2d: 2-2                  [64, 256, 20, 5]          512
│    └─ReLU: 2-3                         [64, 256, 20, 5]          --
├─Sequential: 1-2                        [64, 128, 39, 9]          --
│    └─ConvTranspose2d: 2-4              [64, 128, 39, 9]          32,768
│    └─BatchNorm2d: 2-5                  [64, 128, 39, 9]          256
│    └─ReLU: 2-6                         [64, 128, 39, 9]          --
├─Sequential: 1-3                        [64, 64, 77, 17]          --
│    └─ConvTranspose2d: 2-7              [64, 64, 77, 17]          8,192
│    └─BatchNorm2d: 2-8                  [64, 64, 77, 17]          128
│    └─ReLU: 2-9                         [64, 64, 77, 17]         

In [13]:
# torch.save(netG, "netG.pt")
# netD.load_state_dict()

params = netG.state_dict()

val = torch.empty(batch_size, nz, 20, 5, dtype = torch.float32)

torch.onnx.export(netG, val.to(device), "netG.onnx")

Discriminator Code



In [14]:
# 구분자 코드

class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.conv1 = nn.Sequential(
            nn.Conv2d(nc, ndf, 2, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(ndf, ndf * 2, 2, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv3 = nn.Sequential(
            nn.Conv2d(ndf * 2, ndf * 4, 2, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv4 = nn.Sequential(
            nn.Conv2d(ndf * 4, ndf * 8, 2, 3, 10, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True)
        )

        self.conv5 = nn.Sequential(
            nn.Conv2d(ndf * 8, 2, 3, 1, 0, bias=False),
            nn.Conv2d(2, 1, 5),
        )

        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(2, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        out = self.conv1(input)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)
        # print("out", out.shape)
        out = self.fc(out)
        return out

Now, as with the generator, we can create the discriminator, apply the
``weights_init`` function, and print the model’s structure.




In [15]:
device

'cuda:0'

In [16]:
# Create the Discriminator
netD = Discriminator(ngpu).to(device)

# Handle multi-gpu if desired
if (device == 'cuda' or device == 'mps') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))
    
# Apply the weights_init function to randomly initialize all weights
#  to mean=0, stdev=0.2.
netD.apply(weights_init)

# Print the model
# print(netD)
summary(netD, (batch_size, nz, 20, 5))

Layer (type:depth-idx)                   Output Shape              Param #
Discriminator                            [64, 1]                   --
├─Sequential: 1-1                        [64, 64, 11, 3]           --
│    └─Conv2d: 2-1                       [64, 64, 11, 3]           51,456
│    └─LeakyReLU: 2-2                    [64, 64, 11, 3]           --
├─Sequential: 1-2                        [64, 128, 6, 2]           --
│    └─Conv2d: 2-3                       [64, 128, 6, 2]           32,768
│    └─BatchNorm2d: 2-4                  [64, 128, 6, 2]           256
│    └─LeakyReLU: 2-5                    [64, 128, 6, 2]           --
├─Sequential: 1-3                        [64, 256, 4, 2]           --
│    └─Conv2d: 2-6                       [64, 256, 4, 2]           131,072
│    └─BatchNorm2d: 2-7                  [64, 256, 4, 2]           512
│    └─LeakyReLU: 2-8                    [64, 256, 4, 2]           --
├─Sequential: 1-4                        [64, 512, 8, 7]           --


In [17]:
# torch.save(netD, "netD.pt")

params = netG.state_dict()

val = torch.empty(batch_size, nz, 20, 5, dtype = torch.float32)

torch.onnx.export(netD, val.to(device), "netD.onnx")

In [18]:
# Initialize BCELoss function
criterion = nn.BCELoss()

# Create batch of latent vectors that we will use to visualize
#  the progression of the generator
fixed_noise = torch.randn(batch_size, nz, 20, 5, device=device)

# Establish convention for real and fake labels during training
real_label = 1.
fake_label = 0.

# Setup Adam optimizers for both G and D
optimizerD = optim.NAdam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.NAdam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [19]:
# lis = [audio[0].shape for audio in dataloader]

In [ ]:
# Training Loop

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Loopx...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader):
        
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        netD.zero_grad()
        # Format batch
        real_cpu = data[0].to(device).float()
        b_size = real_cpu.size(0)
        label = data[1]
        label = label.type(torch.FloatTensor).to(device)

        # print("real cpu: ", real_cpu.shape)
        # print("label: ", label.shape)

        # Forward pass real batch through D
        output = netD(real_cpu).view(-1)
        # Calculate loss on all-real batch

        # print("output: ", output.shape)

        errD_real = criterion(output, label)

        # print("errD_real: ", errD_real)

        # Calculate gradients for D in backward pass
        errD_real.backward()
        D_x = output.mean().item()

        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = torch.randn(b_size, nz, 1, 1, device=device)

        # print("noise: ", noise.shape)

        # Generate fake image batch with G
        fake = netG(noise)

        # print("fake", fake.to('cpu').detach().numpy().shape)

        label.fill_(fake_label)
        # Classify all fake batch with D
        output = netD(fake.detach()).view(-1)

        # print("output", output)

        # Calculate D's loss on the all-fake batch
        errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = errD_real + errD_fake
        # Update D
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        # Since we just updated D, perform another forward pass of all-fake batch through D
        output = netD(fake).view(-1)
        # Calculate G's loss based on this output
        errG = criterion(output, label)

        # print("errG", errG)

        # Calculate gradients for G
        errG.backward()
        D_G_z2 = output.mean().item()
        # Update G
        optimizerG.step()
        
        # Output training stats
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        
        # Save Losses for plotting later
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        
        # Check how the generator is doing by saving G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()

        iters += 1

Starting Training Loopx...


K:\Programming\covid_audio\pre_process.py:55: FutureWarning: Pass y=[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.5975768e-08
   3.2500760e-09 8.0286500e-11]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.3877276e-08
   4.1365572e-10 2.1678681e-11]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.1338460e-08
   1.5299034e-09 3.0018416e-10]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.4077438e-17
   9.6076036e-19 3.3780820e-15]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 6.8728776e-20
   9.4312662e-19 3.3103695e-15]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0825610e-18
   1.3132319e-18 3.3355041e-15]]] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.feature.mfcc(np.array(audio), sr=samplerate, n_mels=128)
K:\Programming\covid_audio\pre_process.py:55: FutureWarning: Pass y=[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 2.64370510e-06
   9.40963332e-07 1.04611041e-03]
  [0.00000000e

[0/100][0/60]	Loss_D: 2.0037	Loss_G: 0.5301	D(x): 0.6584	D(G(z)): 0.6511 / 0.5887


K:\Programming\covid_audio\pre_process.py:55: FutureWarning: Pass y=[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 7.1183429e-05
   4.3442713e-05 2.9942922e-03]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 5.9498190e-05
   2.5581665e-04 2.8987587e-03]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 7.9845326e-05
   1.9262962e-04 3.3420790e-03]
  ...
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 2.7105054e-17
   8.7603529e-17 1.9647433e-09]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 6.6353166e-17
   5.8980598e-17 1.9624751e-09]
  [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.4658413e-16
   1.0495077e-16 1.9625921e-09]]] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.feature.mfcc(np.array(audio), sr=samplerate, n_mels=128)
G:\Programdata\envs\covid_audio\lib\site-packages\librosa\util\decorators.py:88: UserWarning: n_fft=2048 is too small for input signal of length=372
  return f(*args, **kwargs)
K:\Pro

[0/100][50/60]	Loss_D: 0.2925	Loss_G: 3.2512	D(x): 0.0512	D(G(z)): 0.0445 / 0.0393


K:\Programming\covid_audio\pre_process.py:55: FutureWarning: Pass y=[[[6.12324058e-08 5.51993615e-08 3.16273827e-07 ... 1.33761232e-05
   1.48080817e-05 7.28263767e-05]
  [1.09767825e-07 8.43722692e-09 3.25079213e-07 ... 8.75548903e-06
   7.83486103e-05 4.18320678e-06]
  [2.31672086e-08 3.92660979e-08 1.20620768e-07 ... 3.41257291e-05
   1.85052904e-05 1.03788327e-04]
  ...
  [7.08769906e-16 2.24853959e-15 8.64820639e-15 ... 7.04731412e-18
   6.03087417e-18 5.23404431e-10]
  [1.26335337e-14 1.35855182e-14 3.16639754e-14 ... 8.51183326e-18
   6.10202535e-18 5.22953847e-10]
  [2.57767135e-14 4.55815771e-14 7.04978136e-14 ... 1.75640752e-17
   8.67361738e-19 5.23037946e-10]]] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  return librosa.feature.mfcc(np.array(audio), sr=samplerate, n_mels=128)
K:\Programming\covid_audio\pre_process.py:55: FutureWarning: Pass y=[[[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.0101862e-05
   2.1220292e-06

[1/100][0/60]	Loss_D: 0.2756	Loss_G: 3.5166	D(x): 0.0414	D(G(z)): 0.0366 / 0.0312
[1/100][50/60]	Loss_D: 0.2308	Loss_G: 4.4642	D(x): 0.0648	D(G(z)): 0.0136 / 0.0117
[2/100][0/60]	Loss_D: 0.2007	Loss_G: 4.3877	D(x): 0.1123	D(G(z)): 0.0204 / 0.0131
[2/100][50/60]	Loss_D: 0.1912	Loss_G: 5.1744	D(x): 0.0495	D(G(z)): 0.0081 / 0.0058
[3/100][0/60]	Loss_D: 0.3846	Loss_G: 4.9002	D(x): 0.0622	D(G(z)): 0.0135 / 0.0085
[3/100][50/60]	Loss_D: 0.2997	Loss_G: 5.6443	D(x): 0.1006	D(G(z)): 0.0055 / 0.0041
[4/100][0/60]	Loss_D: 0.2750	Loss_G: 5.5533	D(x): 0.0523	D(G(z)): 0.0060 / 0.0042
[4/100][50/60]	Loss_D: 0.1689	Loss_G: 5.7511	D(x): 0.0847	D(G(z)): 0.0087 / 0.0033
[5/100][0/60]	Loss_D: 0.2955	Loss_G: 5.3265	D(x): 0.0472	D(G(z)): 0.0062 / 0.0050
[5/100][50/60]	Loss_D: 0.1998	Loss_G: 6.0905	D(x): 0.0775	D(G(z)): 0.0034 / 0.0023
[6/100][0/60]	Loss_D: 0.3176	Loss_G: 5.6712	D(x): 0.0545	D(G(z)): 0.0036 / 0.0035
[6/100][50/60]	Loss_D: 0.3010	Loss_G: 5.8259	D(x): 0.1087	D(G(z)): 0.0042 / 0.0033
[7/100][0/

Results
-------

Finally, lets check out how we did. Here, we will look at three
different results. First, we will see how D and G’s losses changed
during training. Second, we will visualize G’s output on the fixed_noise
batch for every epoch. And third, we will look at a batch of real data
next to a batch of fake data from G.

**Loss versus training iteration**

Below is a plot of D & G’s losses versus training iterations.




In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

**Visualization of G’s progression**

Remember how we saved the generator’s output on the fixed_noise batch
after every epoch of training. Now, we can visualize the training
progression of G with an animation. Press the play button to start the
animation.




Where to Go Next
----------------

We have reached the end of our journey, but there are several places you
could go from here. You could:

-  Train for longer to see how good the results get
-  Modify this model to take a different dataset and possibly change the
   size of the images and the model architecture
-  Check out some other cool GAN projects
   `here <https://github.com/nashory/gans-awesome-applications>`__
-  Create GANs that generate
   `music <https://www.deepmind.com/blog/wavenet-a-generative-model-for-raw-audio/>`__



In [ ]:
# test_data = torch.Tensor(np.array(pd.read_csv("data/test_mfcc_data.csv").iloc[:, 5:]).reshape(-1, 1, 32))

netD(next(iter(test_dataloader)).to(device).float()).view(-1)

In [ ]:
# pd.read_csv("data/train_mfcc_data.csv")

# next(iter(test_dataloader))
answer = []
for i in test_dataloader:
    for j in netD(i.to(device).float()).view(-1):
        answer.append(j.item())

In [ ]:
answer_df = pd.DataFrame(answer)

print(answer_df[answer_df[0] >= 0.39])
answer_df[answer_df[0] >= 0.4] = 1
answer_df[answer_df[0] < 0.4] = 0

In [ ]:
submit = pd.read_csv("data/sample_submission.csv")

submit['covid19'] = answer_df[0]
submit = submit.astype({'covid19': 'int'})
submit.to_csv("submission.csv", index=False)
# submit

In [ ]:
submit[submit['covid19'] >= 0.39]

In [ ]:
df = pd.read_csv("MLP (100, 100, 100).csv")
df[df['covid19'] == 1]